In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
X=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
test=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [4]:
X.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [5]:
test.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
X1=X["full_text"]
t1=test["full_text"]

In [7]:
X1

0       I think that students would benefit from learn...
1       When a problem is a change you have to let it ...
2       Dear, Principal\n\nIf u change the school poli...
3       The best time in life is when you become yours...
4       Small act of kindness can impact in other peop...
                              ...                        
3906    I believe using cellphones in class for educat...
3907    Working alone, students do not have to argue w...
3908    "A problem is a chance for you to do your best...
3909    Many people disagree with Albert Schweitzer's ...
3910    Do you think that failure is the main thing fo...
Name: full_text, Length: 3911, dtype: object

In [8]:
t1

0    when a person has no experience on a job their...
1    Do you think students would benefit from being...
2    Thomas Jefferson once states that "it is wonde...
Name: full_text, dtype: object

In [9]:
X_new=X1.append(t1,ignore_index=True)

In [10]:
X_new

0       I think that students would benefit from learn...
1       When a problem is a change you have to let it ...
2       Dear, Principal\n\nIf u change the school poli...
3       The best time in life is when you become yours...
4       Small act of kindness can impact in other peop...
                              ...                        
3909    Many people disagree with Albert Schweitzer's ...
3910    Do you think that failure is the main thing fo...
3911    when a person has no experience on a job their...
3912    Do you think students would benefit from being...
3913    Thomas Jefferson once states that "it is wonde...
Name: full_text, Length: 3914, dtype: object

In [11]:
tfidf = TfidfVectorizer()
result = tfidf.fit_transform(X_new)

In [12]:
tfidf=result.toarray()

In [13]:
test.shape

(3, 2)

In [14]:
X.shape

(3911, 8)

In [15]:
test=tfidf[len(X1):]
X2=tfidf[:len(X1)]

In [16]:
X2.shape

(3911, 21370)

In [17]:
test.shape

(3, 21370)

In [18]:
IP=X2.shape[1]

In [19]:
IP

21370

In [20]:
X1_train = X.loc[:int(len(X1.index)*0.8)]
y_train = X1_train[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_val = X.loc[int(len(X1.index)*0.8)+1:int(len(X1.index)*0.9)]
y_val = X1_val[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_test = X.loc[int(len(X1.index)*0.9)+1:]
y_test = X1_test[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]

In [21]:
tfidf_train=X2[:int(len(X.index)*0.8)+1]
tfidf_val=X2[int(len(X.index)*0.8)+1:int(len(X.index)*0.9)+1]
tfidf_test=X2[int(len(X.index)*0.9)+1:]

In [22]:
y1_train=y_train["syntax"]
y2_train=y_train["cohesion"]
y3_train=y_train["vocabulary"]
y4_train=y_train["phraseology"]
y5_train=y_train["grammar"]
y6_train=y_train["conventions"]

y1_val=y_val["syntax"]
y2_val=y_val["cohesion"]
y3_val=y_val["vocabulary"]
y4_val=y_val["phraseology"]
y5_val=y_val["grammar"]
y6_val=y_val["conventions"]

y1_test=y_test["syntax"]
y2_test=y_test["cohesion"]
y3_test=y_test["vocabulary"]
y4_test=y_test["phraseology"]
y5_test=y_test["grammar"]
y6_test=y_test["conventions"]

In [23]:
print(tfidf_train.shape)
print(y1_train.shape)
print(tfidf_val.shape)
print(y1_val.shape)
print(tfidf_test.shape)
print(y_test.shape)

(3129, 21370)
(3129,)
(391, 21370)
(391,)
(391, 21370)
(391, 6)


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [25]:
rf1 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf1.fit(tfidf_train, y1_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y1_pred = rf1.predict(tfidf_test)
y1_pred = [round(r,1) for r in y1_pred]


In [26]:
rf2 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf2.fit(tfidf_train, y2_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y2_pred = rf2.predict(tfidf_test)
y2_pred = [round(r,1) for r in y2_pred]



In [27]:
rf3 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf3.fit(tfidf_train, y3_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y3_pred = rf3.predict(tfidf_test)
y3_pred = [round(r,1) for r in y3_pred]

In [28]:
rf4 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf4.fit(tfidf_train, y4_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y4_pred = rf4.predict(tfidf_test)
y4_pred = [round(r,1) for r in y4_pred]

In [29]:
rf5 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf5.fit(tfidf_train, y5_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y5_pred = rf5.predict(tfidf_test)
y5_pred = [round(r,1) for r in y5_pred]

In [30]:
rf6 = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf6.fit(tfidf_train, y6_train)
# pickle.dump(rf, open('Saved_Models/RF_with_PP', 'wb') = [round(r,1) for r in y_pred]
# rf = pickle.load(open('Saved_Models/RF_wth_pp', 'rb'))

y6_pred = rf6.predict(tfidf_test)
y6_pred = [round(r,1) for r in y6_pred]

In [31]:
t1=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [32]:
final=pd.DataFrame()

#final['text_id']=t1["text_id"]
final['syntax']=y1_pred
final['cohesion']=y2_pred
final['vocabulary']=y3_pred
final['phraseology']=y4_pred
final['grammar']=y5_pred
final['conventions']=y6_pred

In [33]:
final

,syntax,cohesion,vocabulary,phraseology,grammar,conventions
0,3.2,3.4,3.5,3.5,3.4,3.2
1,3.0,3.0,3.3,3.2,2.9,2.9
2,3.6,3.4,3.6,3.4,3.3,3.4
3,3.1,3.1,3.1,2.9,3.2,2.8
4,3.0,3.1,3.1,3.1,2.9,3.1
...,...,...,...,...,...,...
386,3.1,3.1,3.3,3.4,3.3,3.2
387,3.4,3.4,3.6,3.3,3.2,3.5
388,2.8,3.0,3.2,3.2,3.1,2.9
389,3.3,3.4,3.5,3.1,3.1,3.3


In [34]:
test.shape

(3, 21370)

In [35]:
test1 = rf1.predict(test).flatten()
test1= [round(r,1) for r in test1]  

test2 = rf2.predict(test).flatten()
test2= [round(r,1) for r in test2]

test3 = rf3.predict(test).flatten()
test3= [round(r,1) for r in test3]

test4 = rf4.predict(test).flatten()
test4= [round(r,1) for r in test4]

test5 = rf5.predict(test).flatten()
test5= [round(r,1) for r in test5]

test6 =rf6.predict(test).flatten()
test6= [round(r,1) for r in test6]

In [36]:
t1=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [37]:
final=pd.DataFrame()

final['text_id']=t1["text_id"]
final['syntax']=test1
final['cohesion']=test2
final['vocabulary']=test3
final['phraseology']=test4
final['grammar']=test5
final['conventions']=test6

In [38]:
final.to_csv('submission.csv', index=False)

In [39]:
final

,text_id,syntax,cohesion,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.1,3.1,3.2,3.0,3.2
1,000BAD50D026,2.8,2.8,3.0,2.9,3.0,2.9
2,00367BB2546B,3.2,3.5,3.6,3.4,3.2,3.4
